# Описание проекта

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Цель - построить модель для определения стоимости. 

# 1. Подготовка данных

In [2]:
#импортируем библиотеки
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
import os
import urllib.request
import warnings 
warnings.filterwarnings('ignore')

import time

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

In [3]:
#загружаем данные
Path('datasets').mkdir(parents=True, exist_ok=True)

def get_file(file_name, url):
    if not os.path.exists(file_name):
        print(file_name, 'файл не найден, будет загружен из сети')
        _=urllib.request.urlretrive(url, file_name)
        
urls = {
    'autos': ('/datasets/autos.csv', 'https://code.s3.yandex.net/datasets/autos.csv')
}

[get_file(*urls[k]) for k in urls]

df = pd.read_csv(urls['autos'][0])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
# посмотрим на распредление данных
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [5]:
# оценим какие пропуски имеются
df.isnull().sum().sort_values(ascending=False)/len(df)

NotRepaired          0.200791
VehicleType          0.105794
FuelType             0.092827
Gearbox              0.055967
Model                0.055606
LastSeen             0.000000
PostalCode           0.000000
NumberOfPictures     0.000000
DateCreated          0.000000
Brand                0.000000
RegistrationMonth    0.000000
Kilometer            0.000000
Power                0.000000
RegistrationYear     0.000000
Price                0.000000
DateCrawled          0.000000
dtype: float64

In [6]:
# удалим признаки, которые не несут в себе никакой информации
df = df.drop(['DateCrawled', 'RegistrationMonth', 'RegistrationYear', 
              'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis=1)

In [7]:
# проверка на дубликаты
df.duplicated().sum()

73748

In [8]:
# удаление повторяющихся наблюдений
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [9]:
# посмотрим на уникальные значения
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    print('Уникальные значения', col, df[col].unique())

Уникальные значения VehicleType [nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
Уникальные значения Gearbox ['manual' 'auto' nan]
Уникальные значения Model ['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'ma

In [10]:
# посмотрим с чем коррелирует цена
df.corr()['Price'].sort_values(ascending=False)

Price        1.000000
Power        0.139003
Kilometer   -0.321205
Name: Price, dtype: float64

- Данные загрузили и изучили: после предобработки оставили 9 признаков и 280 621 объектов
- Выявили и удалили 73 748 дубликатов
- В некоторых признаках содержится от 5 по 20 процентов пропусков - с ними справится сама модель, поэтому заполнять их не стали
- Из датасета убрали признаки, связанные с датами, тк они для обучения модели они не понадобятся и колонку Количество фотографий, тк в нем фактически не содержатся данных

# 2. Обучение моделей

## 2.1. Базовая модель LightGBM

In [11]:
df_gbm = df.copy()
for col in df_gbm.columns:
    col_type = df_gbm[col].dtype
    if col_type == 'object':
        df_gbm[col] = df_gbm[col].astype("category")
        
df_gbm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280621 entries, 0 to 354368
Data columns (total 9 columns):
Price          280621 non-null int64
VehicleType    247978 non-null category
Gearbox        264547 non-null category
Power          280621 non-null int64
Model          263002 non-null category
Kilometer      280621 non-null int64
FuelType       251307 non-null category
Brand          280621 non-null category
NotRepaired    220575 non-null category
dtypes: category(6), int64(3)
memory usage: 10.5 MB


In [12]:
x = df_gbm.drop(['Price'], axis=1)
y = df_gbm['Price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

print('Размер обучающей выборки', x_train.shape)
print('Размер целевого признака', y_train.shape)

Размер обучающей выборки (210465, 8)
Размер целевого признака (210465,)


Подготовили наборы данных для модели LightGBM, а именно: все категориальные признаки переверели из формата object в category, а также разбили имеющиеся данные на обучающую и тестовую выборки в соотношение 75:25

In [25]:
gbm = lgb.LGBMRegressor()

time_start_gbm = time.time()
gbm.fit(x_train, y_train)
time_middle_gbm = time.time()
gbm_pred = gbm.predict(x_test)
time_finish_gbm = time.time()

time_fitting_gbm = time_middle_gbm - time_start_gbm
time_predicting_gbm = time_finish_gbm - time_middle_gbm
RMSE_gbm = mean_squared_error(gbm_pred, y_test) ** 0.5

print('Время обучения:', round(time_fitting_gbm),'сек\nСкорость предсказания:', round(time_predicting_gbm), 'сек')
print('RMSE на тестовой выборке:', round(RMSE_gbm))

Время обучения: 12 сек
Скорость предсказания: 1 сек
RMSE на тестовой выборке: 2440


## 2.2. Модель LightGBM с настроенными гиперпараметрами

In [19]:
%%time
hyper_params = {"max_depth": [int(x) for x in np.linspace(10, 100, num = 33)],
                "num_leaves": [i for i in range(10, 50, 16)]}

fit_params = {"eval_metric" : 'neg_mean_squared_error', 
            "eval_set" : [(x_test,y_test)]}           
                
gbm = lgb.LGBMRegressor(**hyper_params)
gbm = GridSearchCV(gbm, param_grid = hyper_params, cv = 3, verbose=2)
gbm.fit(x_train, y_train, **fit_params)
gbm.best_params_

Fitting 3 folds for each of 99 candidates, totalling 297 fits
[CV] max_depth=10, num_leaves=10 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[1]	valid_0's l2: 1.96561e+07
[2]	valid_0's l2: 1.80619e+07
[3]	valid_0's l2: 1.67524e+07
[4]	valid_0's l2: 1.56417e+07
[5]	valid_0's l2: 1.4678e+07
[6]	valid_0's l2: 1.38544e+07
[7]	valid_0's l2: 1.30754e+07
[8]	valid_0's l2: 1.24299e+07
[9]	valid_0's l2: 1.18846e+07
[10]	valid_0's l2: 1.13725e+07
[11]	valid_0's l2: 1.0926e+07
[12]	valid_0's l2: 1.05493e+07
[13]	valid_0's l2: 1.02174e+07
[14]	valid_0's l2: 9.92341e+06
[15]	valid_0's l2: 9.6531e+06
[16]	valid_0's l2: 9.42338e+06
[17]	valid_0's l2: 9.20232e+06
[18]	valid_0's l2: 9.00245e+06
[19]	valid_0's l2: 8.83448e+06
[20]	valid_0's l2: 8.67736e+06
[21]	valid_0's l2: 8.5453e+06
[22]	valid_0's l2: 8.41191e+06
[23]	valid_0's l2: 8.30396e+06
[24]	valid_0's l2: 8.20345e+06
[25]	valid_0's l2: 8.1142e+06
[26]	valid_0's l2: 8.01884e+06
[27]	valid_0's l2: 7.93989e+06
[28]	valid_0's l2: 7.85294e+06
[29]	valid_0's l2: 7.79524e+06
[30]	valid_0's l2: 7.72965e+06
[31]	valid_0's l2: 7.67216e+06
[32]	valid_0's l2: 7.61925e+06
[33]	valid_0's l2: 7.5

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.4s remaining:    0.0s


[1]	valid_0's l2: 1.96531e+07
[2]	valid_0's l2: 1.80384e+07
[3]	valid_0's l2: 1.67139e+07
[4]	valid_0's l2: 1.55929e+07
[5]	valid_0's l2: 1.46176e+07
[6]	valid_0's l2: 1.38121e+07
[7]	valid_0's l2: 1.30831e+07
[8]	valid_0's l2: 1.23967e+07
[9]	valid_0's l2: 1.18381e+07
[10]	valid_0's l2: 1.13332e+07
[11]	valid_0's l2: 1.08974e+07
[12]	valid_0's l2: 1.05256e+07
[13]	valid_0's l2: 1.01833e+07
[14]	valid_0's l2: 9.90026e+06
[15]	valid_0's l2: 9.64601e+06
[16]	valid_0's l2: 9.41415e+06
[17]	valid_0's l2: 9.21737e+06
[18]	valid_0's l2: 9.01624e+06
[19]	valid_0's l2: 8.84789e+06
[20]	valid_0's l2: 8.70446e+06
[21]	valid_0's l2: 8.56526e+06
[22]	valid_0's l2: 8.44145e+06
[23]	valid_0's l2: 8.33226e+06
[24]	valid_0's l2: 8.22716e+06
[25]	valid_0's l2: 8.13965e+06
[26]	valid_0's l2: 8.04203e+06
[27]	valid_0's l2: 7.97086e+06
[28]	valid_0's l2: 7.89578e+06
[29]	valid_0's l2: 7.83926e+06
[30]	valid_0's l2: 7.77812e+06
[31]	valid_0's l2: 7.72035e+06
[32]	valid_0's l2: 7.66953e+06
[33]	valid_0's l2

[Parallel(n_jobs=1)]: Done 297 out of 297 | elapsed: 60.9min finished


[1]	valid_0's l2: 1.91976e+07
[2]	valid_0's l2: 1.72378e+07
[3]	valid_0's l2: 1.56295e+07
[4]	valid_0's l2: 1.42722e+07
[5]	valid_0's l2: 1.31513e+07
[6]	valid_0's l2: 1.21989e+07
[7]	valid_0's l2: 1.14021e+07
[8]	valid_0's l2: 1.07323e+07
[9]	valid_0's l2: 1.01702e+07
[10]	valid_0's l2: 9.67944e+06
[11]	valid_0's l2: 9.26963e+06
[12]	valid_0's l2: 8.90028e+06
[13]	valid_0's l2: 8.59658e+06
[14]	valid_0's l2: 8.3334e+06
[15]	valid_0's l2: 8.10272e+06
[16]	valid_0's l2: 7.90996e+06
[17]	valid_0's l2: 7.73264e+06
[18]	valid_0's l2: 7.58226e+06
[19]	valid_0's l2: 7.45593e+06
[20]	valid_0's l2: 7.33906e+06
[21]	valid_0's l2: 7.23818e+06
[22]	valid_0's l2: 7.15067e+06
[23]	valid_0's l2: 7.0652e+06
[24]	valid_0's l2: 6.99382e+06
[25]	valid_0's l2: 6.92633e+06
[26]	valid_0's l2: 6.86828e+06
[27]	valid_0's l2: 6.81373e+06
[28]	valid_0's l2: 6.76429e+06
[29]	valid_0's l2: 6.7162e+06
[30]	valid_0's l2: 6.67802e+06
[31]	valid_0's l2: 6.63719e+06
[32]	valid_0's l2: 6.60194e+06
[33]	valid_0's l2: 6

{'max_depth': 21, 'num_leaves': 42}

In [24]:
hyper_params = {"max_depth": 21,
                "num_leaves": 42}          
                
gbm_best = lgb.LGBMRegressor(**hyper_params)

time_start_gbm2 = time.time()
gbm_best.fit(x_train, y_train, **fit_params)
time_middle_gbm2 = time.time()
gbm_pred = gbm_best.predict(x_test)
time_finish_gbm2 = time.time()

time_fitting_gbm2 = time_middle_gbm - time_start_gbm
time_predicting_gbm2 = time_finish_gbm - time_middle_gbm
RMSE_gbm2 = mean_squared_error(gbm_pred, y_test) ** 0.5

print('Время обучения:', round(time_fitting_gbm2),'сек\nСкорость предсказания:', round(time_predicting_gbm2), 'сек')
print('RMSE на тестовой выборке:', RMSE_gbm2)

[1]	valid_0's l2: 1.91976e+07
[2]	valid_0's l2: 1.72378e+07
[3]	valid_0's l2: 1.56295e+07
[4]	valid_0's l2: 1.42722e+07
[5]	valid_0's l2: 1.31513e+07
[6]	valid_0's l2: 1.21989e+07
[7]	valid_0's l2: 1.14021e+07
[8]	valid_0's l2: 1.07323e+07
[9]	valid_0's l2: 1.01702e+07
[10]	valid_0's l2: 9.67944e+06
[11]	valid_0's l2: 9.26963e+06
[12]	valid_0's l2: 8.90028e+06
[13]	valid_0's l2: 8.59658e+06
[14]	valid_0's l2: 8.3334e+06
[15]	valid_0's l2: 8.10272e+06
[16]	valid_0's l2: 7.90996e+06
[17]	valid_0's l2: 7.73264e+06
[18]	valid_0's l2: 7.58226e+06
[19]	valid_0's l2: 7.45593e+06
[20]	valid_0's l2: 7.33906e+06
[21]	valid_0's l2: 7.23818e+06
[22]	valid_0's l2: 7.15067e+06
[23]	valid_0's l2: 7.0652e+06
[24]	valid_0's l2: 6.99382e+06
[25]	valid_0's l2: 6.92633e+06
[26]	valid_0's l2: 6.86828e+06
[27]	valid_0's l2: 6.81373e+06
[28]	valid_0's l2: 6.76429e+06
[29]	valid_0's l2: 6.7162e+06
[30]	valid_0's l2: 6.67802e+06
[31]	valid_0's l2: 6.63719e+06
[32]	valid_0's l2: 6.60194e+06
[33]	valid_0's l2: 6

Использован внутренний метод кодировки признаков, настроили базовые гиперпараметры у модели и, в результате, удалось достичь RMSE - 2 399, что чуть лучше базовой модели (2 440) при это время обучения удалось сократить на 4 сек: с 12 до 8 сек

## 2.3. Модель CatBoost

In [27]:
df_cat = df.copy()
df_cat.dropna(subset=['NotRepaired'], inplace=True)
categorical_cols = [c for c in df_cat.columns if df_cat[c].dtype.name == 'object']
for col in categorical_cols:
    df_cat[col] = df_cat[col].astype("str")
    df_cat[col].fillna("unknown")
        
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220575 entries, 1 to 354367
Data columns (total 9 columns):
Price          220575 non-null int64
VehicleType    220575 non-null object
Gearbox        220575 non-null object
Power          220575 non-null int64
Model          220575 non-null object
Kilometer      220575 non-null int64
FuelType       220575 non-null object
Brand          220575 non-null object
NotRepaired    220575 non-null object
dtypes: int64(3), object(6)
memory usage: 16.8+ MB


In [28]:
x = df_cat.drop(['Price'], axis=1)
y = df_cat['Price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

print('Размер обучающей выборки', x.shape)
print('Размер целевого признака', y.shape)

Размер обучающей выборки (220575, 8)
Размер целевого признака (220575,)


Подготовили данные для модели CatBoost: удалили пропуски в признаки NotRepaired, тк эти данные не спрогнозировать и заполнили остальные пропуски новой категорией формата string - Unknown. Далее традиционно разбили фичи на обучающие и целевой.

In [29]:
cat_model = CatBoostRegressor(loss_function='RMSE')

time_start_cat = time.time()
cat_model.fit(x_train, y_train, cat_features = categorical_cols)
time_middle_cat = time.time()
cat_pred = cat_model.predict(x_test)
time_finish_cat = time.time()

time_fitting_cat = time_middle_cat - time_start_cat
time_predicting_cat = time_finish_cat - time_middle_cat
RMSE_cat = mean_squared_error(cat_pred, y_test) ** 0.5

print('Время обучения:', round(time_fitting_cat) / 60,'мин\nСкорость предсказания:', round(time_predicting_cat), 'сек')
print('RMSE на тестовой выборке:', RMSE_cat)

0:	learn: 4710.7970392	total: 582ms	remaining: 9m 41s
1:	learn: 4641.6430141	total: 1.18s	remaining: 9m 50s
2:	learn: 4574.1480708	total: 1.77s	remaining: 9m 49s
3:	learn: 4511.0387699	total: 2.18s	remaining: 9m 3s
4:	learn: 4450.3568832	total: 2.77s	remaining: 9m 11s
5:	learn: 4390.6312785	total: 3.28s	remaining: 9m 2s
6:	learn: 4332.8153572	total: 3.78s	remaining: 8m 55s
7:	learn: 4278.7869381	total: 4.37s	remaining: 9m 2s
8:	learn: 4227.0842175	total: 4.68s	remaining: 8m 35s
9:	learn: 4176.6463620	total: 5.27s	remaining: 8m 41s
10:	learn: 4125.6276777	total: 5.69s	remaining: 8m 31s
11:	learn: 4076.9658575	total: 6.27s	remaining: 8m 36s
12:	learn: 4032.0786135	total: 6.67s	remaining: 8m 26s
13:	learn: 3987.4932905	total: 7.17s	remaining: 8m 25s
14:	learn: 3944.4892173	total: 7.57s	remaining: 8m 16s
15:	learn: 3904.3377411	total: 7.97s	remaining: 8m 10s
16:	learn: 3865.1995409	total: 8.47s	remaining: 8m 9s
17:	learn: 3828.1641307	total: 8.86s	remaining: 8m 3s
18:	learn: 3794.3948026	t

Модель CatBoost обучается дольше (8.2 мин) базовой модели LightGBM и результат RMSE примерно одинаковый - 2 533

## 2.3. Модель линейной регрессии

In [30]:
df_linear = pd.get_dummies(df_cat, drop_first=True)
x = df_linear.drop(['Price'], axis=1)
y = df_linear['Price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

print('Размер обучающей выборки', x.shape)
print('Размер целевого признака', y.shape)

Размер обучающей выборки (220575, 308)
Размер целевого признака (220575,)


Закодировали категориальные переменные с помощью техники прямого кодирования и разбили фичи для обучения модели.

In [31]:
linear_model = LinearRegression()

time_start_linear = time.time()
linear_model.fit(x_train, y_train)
time_middle_linear = time.time()
linear_pred = linear_model.predict(x_test)
time_finish_linear = time.time()

time_fitting_linear = time_middle_linear - time_start_linear
time_predicting_linear = time_finish_linear - time_middle_linear
RMSE_linear = mean_squared_error(linear_pred, y_test) ** 0.5

print('Время обучения:', round(time_fitting_linear),'сек\nСкорость предсказания:', round(time_predicting_linear), 'сек')
print('RMSE на тестовой выборке:', RMSE_linear)

Время обучения: 18 сек
Скорость предсказания: 0 сек
RMSE на тестовой выборке: 3351.8496867901445


Модель обучается быстро (18 сек), но результат RMSE не является лучшим - 3 351

# 3. Анализ моделей

In [32]:
analysis_data = {'LightGBM_Base': [time_fitting_gbm, time_predicting_gbm, RMSE_gbm],
                'LightGBM_HyperParams': [time_fitting_gbm2, time_predicting_gbm2, RMSE_gbm2],
                'CatBoost': [time_fitting_cat, time_predicting_cat, RMSE_cat],
                'LinearRegression': [time_fitting_linear, time_predicting_linear, RMSE_linear]}
analysis = pd.DataFrame(analysis_data)
analysis

,LightGBM_Base,LightGBM_HyperParams,CatBoost,LinearRegression
0,11.632320,8.003739,491.856265,17.616408
1,1.205630,1.096138,0.525007,0.240105
2,2440.188011,2399.854757,2533.316507,3351.849687


Представили в новой таблице сравнение всех моделей:
- Самый лучший RMSE у LightGBM модели с настроенными гиперпараметрами - 2 399 по сравнению с CatBoost (2 533) и Линейной регрессией (3 351)
- При этом он также самый быстрый: модель обучается всего за 8 сек, больше всего времени требуется CatBoost (8 мин), остальные справляются примерно за 15 сек
- У скорости предсказывания самое низкое отклонение: время колеблется от 0.24 сек у Линейной регрессии до 1.2 сек у базовой LightGBM, то есть все модели справились практически одинаково хорошо

# Выводы

В рамках проекта была проделана следующая работа:
- Загружены, изучены и предобработаны данные: удалены дубликаты и оставлены только те признаки, на которых будут обучаться модели: убрали признаки связанные с датой и категориальные переверели из категорийный формат.
- Обучили разные варианты моделей: базовый LightGBM и версия модели с настроенными гиперпараметрами, CatBoost и Линейная регрессия
- Лучший результат по RMSE и времени обучения у LightGBM с максимальной глубиной 21 и количеством листьев 42 - 2 399 и 8 сек. CatBoost близок по RMSE (2 533), но целых 8 минут требуется на обучение и Линейная регрессия работает молниеносно (17 сек), но не с лучшим результатом (3 351)